# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/model/')

In [2]:
import pandas as pd
import numpy as np
import pickle
import itertools
import random
import torch
import optuna
from model.utils import *

/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pipeline_config import *

# Seeds for Comparisons:

In [4]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [10]:
Year

2019

# Read Data

In [5]:
with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

with open(train_dataset_dict_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)

with open(test_dataset_dict_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

# Hierarchical GRU

In [6]:
def objective(trial):
    with open(son_parent_path, 'rb') as f:
        son_parent_dict = pickle.load(f)

    with open(train_dataset_dict_path, 'rb') as f:
        train_dataset_dict = pickle.load(f)

    with open(test_dataset_dict_path, 'rb') as f:
        test_dataset_dict = pickle.load(f)

    with open(category_id_to_category_name_path, 'rb') as f:
        category_id_to_name_dict = pickle.load(f)
        
    with open(categories_per_indent_path, 'rb') as f:
        categories_per_indent_dict = pickle.load(f)

    weights_path = weightspath
    
    #--------------------------------------------------------------------------------------------------------------------------------------#

    loss_coef = trial.suggest_float('loss_coef_1',  1e-10, 1e-1, log=True)
    Lr = trial.suggest_float('Lr', 1e-5, 1e-1, log=True)

    hgru_models = {}
    num_categories = 0

    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                parent_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
                parent_optimizer = torch.optim.AdamW(parent_model.parameters(), lr=Lr)
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            min_error = training_and_evaluation(trial, model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)
            hgru_models[category] = min_error

    average_error = sum(list(hgru_models.values()))/len(list(hgru_models.values()))
    return average_error


In [7]:
EPOCHS = 30
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2023-05-26 01:00:14,037] A new study created in memory with name: no-name-3e1a4c02-92a8-4011-96a4-697d6a6c1d54


num categories: 1
category id|name: 2|All-items
num categories: 2
category id|name: 256|Alcoholic beverages, tobacco products and recreational cannabis


/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/

num categories: 3
category id|name: 290|All-items excluding alcoholic beverages, tobacco products and smokers' supplies and recreational cannabis
num categories: 4
category id|name: 287|All-items excluding energy
num categories: 5
category id|name: 284|All-items excluding food
num categories: 6
category id|name: 285|All-items excluding food and energy
num categories: 7
category id|name: 302|All-items excluding gasoline
num categories: 8
category id|name: 289|All-items excluding mortgage interest cost
num categories: 9
category id|name: 293|All-items excluding shelter
num categories: 10
category id|name: 139|Clothing and footwear
num categories: 11
category id|name: 288|Energy
num categories: 12
category id|name: 3|Food
num categories: 13
category id|name: 286|Food and energy
num categories: 14
category id|name: 201|Health and personal care
num categories: 15
category id|name: 96|Household operations, furnishings and equipment
num categories: 16
category id|name: 219|Recreation, educati

[I 2023-05-26 01:47:28,848] Trial 0 finished with value: 0.2044547529353318 and parameters: {'loss_coef_1': 0.011591786104529891, 'Lr': 0.08370957582295632}. Best is trial 0 with value: 0.2044547529353318.


...

num categories: 1
category id|name: 2|All-items
num categories: 2
category id|name: 256|Alcoholic beverages, tobacco products and recreational cannabis


/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.11/lib/

num categories: 3
category id|name: 290|All-items excluding alcoholic beverages, tobacco products and smokers' supplies and recreational cannabis
num categories: 4
category id|name: 287|All-items excluding energy
num categories: 5
category id|name: 284|All-items excluding food
num categories: 6
category id|name: 285|All-items excluding food and energy
num categories: 7
category id|name: 302|All-items excluding gasoline
num categories: 8
category id|name: 289|All-items excluding mortgage interest cost
num categories: 9
category id|name: 293|All-items excluding shelter
num categories: 10
category id|name: 139|Clothing and footwear
num categories: 11
category id|name: 288|Energy
num categories: 12
category id|name: 3|Food
num categories: 13
category id|name: 286|Food and energy
num categories: 14
category id|name: 201|Health and personal care
num categories: 15
category id|name: 96|Household operations, furnishings and equipment
num categories: 16
category id|name: 219|Recreation, educati

[W 2023-05-26 17:18:44,958] Trial 21 failed with parameters: {'loss_coef_1': 9.874951458519302e-08, 'Lr': 0.09423857369400652} because of the following error: RuntimeError('File /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/models_weights/All other passenger vehicle operating expenses.pt cannot be opened.').
Traceback (most recent call last):
  File "/Users/mvilenko/Library/Python/3.11/lib/python/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_71469/4198739395.py", line 51, in objective
    min_error = training_and_evaluation(trial, model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: File /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/models_weights/All other passenger vehicle operating expenses.pt cannot be opened.

In [ ]:
best_trial = study.best_trial
best_trial

In [ ]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model = Model
        basic_optimizer = Optimizer
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [ ]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [ ]:
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

In [ ]:
with open(test_predictions_path, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Get Best Model Weights:

In [ ]:
#def get_weights_per_category(category_list, dir_path):
#    weights = {}
#    for category in category_list:
#        model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
#        model.to(device)
#        optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
        
#        best_checkpoint_path = dir_path+category + '.pt'

#        category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(best_checkpoint_path, model, optimizer)
#        category_model_weights = unify_model_weights(category_model)

#        weights[category] = category_model_weights
    
#    return weights

In [ ]:
def get_weights_per_category(category_id_list, dir_path):
    basic_model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
    basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
    #basic_model.to(device)

    best_models_weights_dict = {}

    for category_id in category_id_list:
        category_name = category_id_to_name_dict[category_id]
        ckp_path = dir_path+category_name+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        category_model_weights = unify_model_weights(model)
        best_models_weights_dict[category_id] = category_model_weights
        
    return best_models_weights_dict

In [ ]:
dir_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/models_weights/'

In [ ]:
category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])


In [ ]:
len(categories)

In [ ]:
len(category_id_list)

In [ ]:
weights_dict = get_weights_per_category(category_id_list, dir_path)

In [ ]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/hgru_model_weights.pickle', 'wb') as handle:
    pickle.dump(weights_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)